In [ ]:
from transformers import pipeline, AutoTokenizer, CamembertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from labels import LABELS

# Définition du modèle

In [ ]:
# https://camembert-model.fr/posts/tutorial/#:~:text=Il%20faut%20enfin%20s'assurer,cette%20limite%20de%20512%20tokens.
model_name = "mtheo/camembert-base-xnli"
candidate_labels=list(map(lambda x: x.lower(), LABELS))


In [ ]:
import pandas as pd

data = "./dataset_resume.csv"

df = pd.read_csv(data)
df.head(3)

# clean

In [ ]:
# df_new = df.copy()
# for label in LABELS:
#   df_new.insert(df_new.shape[1], label, pd.Series(0, dtype=int, index=range(df_new.shape[0])))

# for row in df_new.iterrows():
#   for cat in ['cat1', 'cat2', 'cat3']:
#     for cat_col in LABELS:
#       if row[1].loc[cat] == cat_col:
#         df_new.at[row[0], cat_col] = 1
#         break

# df_new.drop(columns=['cat1', 'cat2', 'cat3', 'url'], inplace=True)
# df_new = df_new.iloc[:, 1:]
# df_new.head(2)

In [ ]:
# print(df_new.description.isnull().sum())

# df_new.dropna(subset=['description'], inplace=True)

# split

In [ ]:
df_new = df.copy()

for index, row in df_new.iterrows():
    df_new.loc[index, "description"] = (
            row.resume
            if isinstance(row.resume, str)
            else row.description
        )

In [ ]:
train_df, val_df = train_test_split(df_new, test_size=.2, shuffle=True, random_state=42)
#train_df[LABELS].sum().sort_values().plot(kind='barh')
#val_df[LABELS].sum().sort_values().plot(kind='barh')

plt.barh(np.arange(len(LABELS)) + .2, train_df[LABELS].sum(), 0.4, label = 'Train')
plt.barh(np.arange(len(LABELS)) - .2, val_df[LABELS].sum(), 0.4, label = 'Val')

plt.yticks(np.arange(len(LABELS)), LABELS)
plt.xlabel("NB")
plt.ylabel("Label")
plt.legend()
plt.show()


Pour savoir le nombre de token dans les descriptions

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
token_counts = []

for _, row in train_df.iterrows():
  token_count = len(tokenizer.encode(
    row["description"],
    max_length=512,
    truncation=False
  ))
  token_counts.append(token_count)

sns.histplot(token_counts)
plt.xlim([0, 512])

print((train_df["description"].apply(lambda x: len(x)) > 512).any())

In [ ]:
import os
from dataset import EventDataModule

os.environ["TOKENIZERS_PARALLELISM"] = "false"

data_module = EventDataModule(df_new, batch_size=1)
data_module.setup()

# train_dataset = EventDataset(df_new, tokenizer=tokenizer, max_token_len=512)

# sample_item = train_dataset[1]
# display(sample_item.keys())

# print(sample_item['description_text'])

# print(train_dataset.tokenized_str(sample_item['input_ids']))

In [ ]:
# for batch in data_module.train_dataloader():
#     print(batch)
#     break

In [ ]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from event_model import EventClassifier

model = EventClassifier(len(LABELS))
trainer = L.Trainer(max_epochs=3, callbacks=[
        EarlyStopping(monitor="val_loss", patience=4, mode="min"),
        ModelCheckpoint(dirpath='./'),
    ])
trainer.fit(model, data_module)

# test brut
zero-shot-classification

In [ ]:
str_test1 = df.loc[0]["description"]
str_test2 = df.loc[146]["description"]

str_test = str_test2

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = CamembertForSequenceClassification.from_pretrained(model_name)

classifier = pipeline(task="zero-shot-classification", model=model, tokenizer=tokenizer)
res = classifier(str_test, candidate_labels=candidate_labels)

# t = tokenizer(str_test)
# print(tokenizer.tokenize(str_test))
display(res)